In [ ]:
import cv2
import math
import argparse
from deepface import DeepFace 
import json
import matplotlib.pyplot as plt

photo=[("Database/1.jpeg"),("Database/2.jpeg"),("Database/3.jpg"),("Database/4.jpeg"),("Database/5.jpg"),("Database/6.jpg"),
      ("Database/7.jpg"),("Database/8.jpg"),("Database/9.jpg"),("Database/10.jpg"),("Database/11.jpg"),("Database/12.jpg"),
      ("Database/13.jpg"),("Database/14.jpg"),("Database/15.jpg"),("Database/16.jpg"),("Database/17.jpg"),("Database/18.jpg"),
      ("Database/19.jpg"), ("Database/20.jpg"),("Database/21.jpg"),("Database/22.jpg"),("Database/23.jpg"),("Database/24.jpg"),
      ("Database/25.jpg")]

a=0
try:

    for x in range(len(photo)):

        print(photo[a])

        def yuzTanimi(net, frame, conf_threshold=0.7):
            frameOpencvDnn=frame.copy()
            frameHeight=frameOpencvDnn.shape[0]
            frameWidth=frameOpencvDnn.shape[1]
            blob=cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

            net.setInput(blob)
            detections=net.forward()
            faceBoxes=[]
            for i in range(detections.shape[2]):
                confidence=detections[0,0,i,2]
                if confidence>conf_threshold:
                    x1=int(detections[0,0,i,3]*frameWidth)
                    y1=int(detections[0,0,i,4]*frameHeight)
                    x2=int(detections[0,0,i,5]*frameWidth)
                    y2=int(detections[0,0,i,6]*frameHeight)
                    faceBoxes.append([x1,y1,x2,y2])
                    cv2.rectangle(frameOpencvDnn, (x1,y1), (x2,y2), (0,255,0), int(round(frameHeight/150)), 8)
            return frameOpencvDnn,faceBoxes

        parser = argparse.ArgumentParser()

        parser.add_argument('--image')

        args = parser.parse_args('')

        img = cv2.imread(photo[a]) 
        color_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        prediction = DeepFace.analyze(color_img)

        resultImg=photo[a]

        faceProto="opencv_face_detector.pbtxt"
        faceModel="opencv_face_detector_uint8.pb"
        ageProto="age_deploy.prototxt"
        ageModel="age_net.caffemodel"
        genderProto="gender_deploy.prototxt"
        genderModel="gender_net.caffemodel"

        MODEL_MEAN_VALUES=(78.4263377603, 87.7689143744, 114.895847746)
        ageList=['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
        genderList=['Male','Female']

        faceNet=cv2.dnn.readNet(faceModel,faceProto)
        ageNet=cv2.dnn.readNet(ageModel,ageProto)
        genderNet=cv2.dnn.readNet(genderModel,genderProto)

        args.image=photo[a]

        video=cv2.VideoCapture(args.image if args.image else 0)
        padding=20
        while cv2.waitKey(1)<0 :
            hasFrame,frame=video.read()
            if not hasFrame:
                cv2.waitKey()
                break

            resultImg,faceBoxes=yuzTanimi(faceNet,frame)
            if not faceBoxes:
                print("Yüz algılanamadı")

            for faceBox in faceBoxes:
                face=frame[max(0,faceBox[1]-padding):
                           min(faceBox[3]+padding,frame.shape[0]-1),max(0,faceBox[0]-padding)
                           :min(faceBox[2]+padding, frame.shape[1]-1)]

                blob=cv2.dnn.blobFromImage(face, 1.0, (227,227), MODEL_MEAN_VALUES, swapRB=False)
                genderNet.setInput(blob)
                genderPreds=genderNet.forward()
                gender=genderList[genderPreds[0].argmax()]
                print(f'Gender: {gender}')

                ageNet.setInput(blob)
                agePreds=ageNet.forward()
                age=ageList[agePreds[0].argmax()]
                print(f'Age: {age[1:-1]} years')
                print(f'Race: {prediction["dominant_race"]}')


                cv2.putText(resultImg, f'{gender}, {age},{prediction["dominant_race"]}', (faceBox[0], faceBox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 2, cv2.LINE_AA)
                cv2.imshow("Yuz Parametreleri ile Yas,Cinsiyet ve Irk Tahmini", resultImg)

        a=a+1
    
except:
    print("Hatalı Yükleme")

Action: age:  25%|█████████████████▊                                                     | 1/4 [00:00<00:00,  8.32it/s]

Resimler/26.jpg


Action: race: 100%|██████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  5.84it/s]


Gender: Male
Age: 25-32 years
Race: middle eastern
